# Introduction

This notebook assumes the previous notebook has been completed, which establishes a DuckDB database with a "World" and "Cities" table, each with geographical attributes.

Let's now connect with DuckDB to this existing 'world-cities' database (make sure you stopped your previous notebook so that the database file is not locked):

In [0]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [0]:
%sql duckdb:///world_cities.db

In [0]:
%sql SET python_scan_all_frames=true

In [0]:
%config SqlMagic.autopandas = True

In [0]:
%sql LOAD spatial;

# Spatial Functions

DuckDB's `spatial` extension contains many additional functions, which are detailed in the [DuckDB documentation](https://duckdb.org/docs/stable/core_extensions/spatial/functions).

## Areas

Let's walk through an example, by calculating the area of each region. This can be achieved using the DuckDB's `ST_Area()` function. Below would therefore be a query that returns the **five largest countries in Africa**.

In [0]:
%%sql
SELECT name AS country, ST_Area(geom) AS total_area
  FROM World
 WHERE continent='Africa'
 ORDER BY total_area DESC
 LIMIT 5

Now there is a small catch with area calculations, which is alluded to by the low area sizes returned (these reflect the geometry type and selected SRID). 

In order to get the size in square meters ($m^2$), we need use the **ST_Area_Spheroid** function which treats polygons not as objects on a flat plane, but on a sphere, which makes them suitable for geodetic data like these world maps here:

In [0]:
%%sql
SELECT name AS country, ST_Area_Spheroid(geom) AS total_area_m2
  FROM World
 WHERE continent='Africa'
 ORDER BY total_area_m2 DESC
 LIMIT 5

Final minor addition if scientific notation isn't ideal - we can express this as a complete integer using a basic Pandas transformation.

In [1]:
%%sql
results << SELECT name AS country, ST_Area_Spheroid(geom) AS total_area_m2
  FROM World
 WHERE continent='Africa'
 ORDER BY total_area_m2 DESC
 LIMIT 5

In [0]:
results['total_area_m2'] = results['total_area_m2'].astype('int64')
results

## Points

Let's consider another spatial function - `ST_Contains()`. This can be used to determine if a given point lies within a region.

According to Google, the coordinates of Paris are `48.8566° N, 2.3522° E`. If we store this as a geometric point (using `ST_Point` - note the longitude comes first!), we can then see if any of the polygons in our countries dataframe contain this point. As expected, this should yield France:

In [0]:
%%sql
 SELECT name
   FROM World
  WHERE ST_Contains(geom, ST_Point(2.3, 48.86))

### Task Wk6-1

**Task: Find all cities in the dataset located within Australia.**

Hint: this involves using the ST functions as the *join* condition. As an extension, see if it can be achieved with multiple different spatial functions (reminder to consider the [DuckDB docs for more functions](https://duckdb.org/docs/stable/core_extensions/spatial/functions)).

In [0]:
%%sql
--## EXAMPLE SOLUTION
SELECT A.name, B.city, B.capital
  FROM World A JOIN Cities B ON ST_Contains(A.geom, B.geom)  /* ST_Within(B.geom, A.geom) */  /* ST_Intersects(A.geom, B.geom) */
 WHERE A.name = 'Australia'

### Task Wk6-2

**Task: List all countries that have more than two primary capital cities.**

Which appears to have the most? Is this accurate? Why or why not?

In [0]:
%%sql
--## EXAMPLE SOLUTION
SELECT A.name, count(*) cities
  FROM World A JOIN Cities B ON ST_Contains(A.geom, B.geom)
 WHERE capital = 'primary'
 GROUP BY A.name
HAVING count(*) > 2
 ORDER BY cities DESC

## Intersections

Returning to our example of Kosovo earlier, let's see if we can determine it's neighbouring countries.

We can leverage a **self-join** with the 'world' table to determine which countries share a boundary with each other. Normally, we would expect to use the `ST_Touch()` function for this, but in our dataset, the boundaries are in low-resolution, so some border shapes actually intersect. Hence the more generic `ST_Intersects()` spatial relationship function should suffice.

In [0]:
%%sql
SELECT B.name
  FROM World A JOIN World B ON ST_Intersects(A.geom, B.geom)
 WHERE A.name = 'Kosovo' AND B.name != A.name

### Task Wk6-3

**Task: Find all countries that neighbour Germany, ordered by the furthest north neighbour to the nearest south.**

Ordering successfully will require the y-value of the centre point of a given polygon - try finding [spatial functions](https://duckdb.org/docs/stable/core_extensions/spatial/functions) to achieve this!

In [0]:
%%sql
--## EXAMPLE SOLUTION
SELECT B.name
  FROM World A JOIN World B ON (ST_Intersects(A.geom, B.geom))
 WHERE A.name = 'Germany' AND B.name != A.name
 ORDER BY ST_Y(ST_Centroid(B.geom)) DESC

### Task Wk6-4

**Task: Find the most common city name in the dataset. Afterwards, determine which countries these cities exist in.**

There are 4 'San Luis' cities - 2 in Cuba, 1 in Argentina, and 1 in Guatemala. Are any city names more common, and what countries are they found in?

Spoiler: you can confirm your answer using [this Wikipedia page](https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Colombia).

In [0]:
%%sql
--## EXAMPLE SOLUTION - query 1
SELECT city, count(*)
  FROM Cities
 GROUP BY city
HAVING count(*) > 1
 ORDER BY count(*) DESC
 LIMIT 5

In [0]:
%%sql
--## EXAMPLE SOLUTION - query 2
SELECT A.name, count(*) cities
  FROM World A JOIN Cities B ON ST_Contains(A.geom, B.geom)
 WHERE B.city = 'La Union'
 GROUP BY A.name
 ORDER BY cities DESC

Close the connection to the "world_cities" database file:

In [0]:
%sql --close duckdb:///world_cities.db